In [1]:
# !pip install pytesseract

In [2]:
# PYTHON IMPORTS
import os, copy, math, re
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image

# DATA IMPORTS 
import random, h5py, glob
import numpy as np
import requests
import pandas as pd

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# SHAPES IMPORTS
import shapely
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import LineString, Polygon, Point

# MY OWN CLASSES
from FindGrid import *

# OCR libraries
from fuzzywuzzy import fuzz, process

# PREFERENCES
Image.MAX_IMAGE_PIXELS = 933120000

C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Download Places Datasets

In [3]:
if False:
    # Base URL pattern with a placeholder for the integer
    base_url = "https://www2.census.gov/geo/tiger/TIGER2022/PLACE/tl_2022_{:02d}_place.zip"  # Replace with your base URL
    savedir  = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\ReferenceDatasets\Places"
    # Loop through integers from 1 to 100
    for i in range(1, 101):
        # Construct the URL by formatting the integer into the base URL
        url = base_url.format(i)

        try:
            # Send an HTTP GET request to the URL
            response = requests.get(url)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Save the response content to a file with a filename based on the integer
                filename = f"{i}.zip"  # You can change the filename format
                with open(f"{savedir}/{filename}", "wb") as file:
                    file.write(response.content)
                print(f"Saved {filename}")
            else:
                print(f"Failed to retrieve {url}. Status code: {response.status_code}")
        except Exception as e:
            print(f"An error occurred while processing {url}: {str(e)}")
            
def strip_non_numeric_and_convert(field_value):
    return pd.to_numeric(field_value.str.replace(r'[^0-9]', ''))

Read Data and Preprocess

In [4]:
data_dir = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/"

places_directory = f"{data_dir}ReferenceDatasets/Places"

CIDs        = pd.read_csv(f"{data_dir}ReferenceDatasets/CommunityNumbers.csv")
CIDs["CID"] = strip_non_numeric_and_convert(CIDs["CID"])
CIDs["ST"]  = np.floor(CIDs["CID"] / 1e4)

counties    = gpd.read_file(f"{data_dir}ReferenceDatasets/Counties/tl_2019_us_county.shp")
counties['STATEFP'] = np.array(counties['STATEFP']).astype(np.int32)

C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: The default value of regex will change from True to False in a future version.


In [5]:
# Initialize an empty dictionary to store the GeoDataFrames
gdf_dict = {}

# Loop through subdirectories
for subdirectory_name in tqdm(os.listdir(places_directory)):
    subdirectory_path = os.path.join(places_directory, subdirectory_name)
    
    # Check if the item in the main directory is a directory
    if os.path.isdir(subdirectory_path):
        
        # List shapefiles in the subdirectory (assuming they all have ".shp" extensions)
        shapefile_list = [f for f in os.listdir(subdirectory_path) if f.endswith('.shp')]

        if shapefile_list:
            # Assuming there's only one shapefile per subdirectory
            shapefile_name = shapefile_list[0]

            # Load the shapefile into a GeoDataFrame
            gdf = gpd.read_file(os.path.join(subdirectory_path, shapefile_name))

            # Add the GeoDataFrame to the dictionary with the subdirectory name as the key
            gdf_dict[int(subdirectory_name)] = gdf

  0%|          | 0/112 [00:00<?, ?it/s]

Fuzzy matching between DataFrame keys

In [6]:
# Function to find the best fuzzy match
def find_best_match(left_value, choices):
    return process.extract(left_value, choices, scorer=fuzz.ratio, limit=1)

county_options = counties['NAMELSAD'].to_numpy()

# Apply the fuzzy matching and merge the DataFrames
merged_data = []

for idx, left_row in tqdm(CIDs.iterrows(), total=CIDs.shape[0]):
    if type(left_row["County"]) == str:
        index = counties['STATEFP'] == left_row["ST"]
        index = index.to_numpy()
        best_match = find_best_match(left_row["County"], county_options[index].tolist())
        if len(best_match) == 0:
            continue
        merged_data.append([left_row["County"], best_match[0][0], left_row["ST"]])

  0%|          | 0/25512 [00:00<?, ?it/s]

Create DataFrame for merges and drop duplicates

In [7]:
merged_df = pd.DataFrame(merged_data, columns=['CIDs', 'Counties', "ST"]) 

merged_df[['CIDs', 'Counties']] = merged_df[['CIDs', 'Counties']].astype(pd.StringDtype())
merged_df = merged_df.drop_duplicates()

Convert to Strings

In [8]:
CIDs['CID'] = CIDs['CID'].astype(pd.StringDtype())
counties['NAMELSAD'] = counties['NAMELSAD'].astype(pd.StringDtype())

Merge DataFrames and add Geometry

In [9]:
result_df = pd.merge(CIDs, merged_df, right_on=['CIDs', 'ST'], left_on=['County', 'ST'], how='inner')
result_df = pd.merge(result_df, counties, right_on=['NAMELSAD', 'STATEFP'], left_on=['Counties', 'ST'], how='inner')
result_df = gpd.GeoDataFrame(result_df, geometry=result_df["geometry"])

In [10]:
result_df = result_df[["CID", "Community Name", "County", "STATEFP", "COUNTYFP" , "GEOID", "Counties",]].copy()

In [11]:
list_dicts = {}
for k, v in gdf_dict.items():
    list_dicts[k] = gdf_dict[k]["NAMELSAD"].to_list()

In [24]:
# Function to find the best fuzzy match
def find_best_match(left_value, choices):
    return process.extract(left_value, choices, scorer=fuzz.ratio, limit=1)

county_options = counties['NAMELSAD'].to_numpy()

# Apply the fuzzy matching and merge the DataFrames
merged_data = []

scores = []

for idx, left_row in tqdm(result_df.iterrows(), total=result_df.shape[0]):
    if type(left_row["Community Name"]) == str:
        if "county" not in left_row["Community Name"].lower():
        
            places_list = list_dicts[int(left_row["STATEFP"])]

            best_match = find_best_match(left_row["Community Name"], places_list)


            if len(best_match) == 0:
                scores.append(0)
                continue

            scores.append(best_match[0][1])    

            merged_data.append([left_row["CID"], best_match[0][0]])
        else:
            scores.append(0)

  0%|          | 0/24806 [00:00<?, ?it/s]

In [ ]:
len(scores)

In [25]:
merged_df.shape

(24806, 2)

In [26]:
merged_df = pd.DataFrame(merged_data, columns=['CID', "NAMELSAD"]) 

merged_df[['NAMELSAD']] = merged_df[["NAMELSAD"]].astype(pd.StringDtype())
merged_df = merged_df.drop_duplicates()

In [27]:
# Convert the dictionary values (GeoDataFrames) into a list
gdf_list = list(gdf_dict.values())

# Merge all GeoDataFrames into a single large GeoDataFrame
places_dfs = pd.concat(gdf_list, ignore_index=True)
places_dfs["STATEFP"] = places_dfs["STATEFP"].astype(np.int32)
result_df["STATEFP"] = result_df["STATEFP"].astype(np.int32)

In [28]:
print(result_df.shape)
result_df_1 = pd.merge(result_df, merged_df, right_on=['CID'], left_on=['CID'], how='left')
result_df_1['MATCH_SCORE'] = scores
print(result_df_1.shape)
result_df_1 = pd.merge(result_df_1, places_dfs.add_suffix("_p"), right_on=['NAMELSAD_p', 'STATEFP_p'], left_on=['NAMELSAD',"STATEFP"], how='left')
print(result_df_1.shape)

(24806, 7)
(24806, 9)
(24863, 26)


In [29]:
result_df_1.keys()

Index(['CID', 'Community Name', 'County', 'STATEFP', 'COUNTYFP', 'GEOID',
       'Counties', 'NAMELSAD', 'MATCH_SCORE', 'STATEFP_p', 'PLACEFP_p',
       'PLACENS_p', 'GEOID_p', 'NAME_p', 'NAMELSAD_p', 'LSAD_p', 'CLASSFP_p',
       'PCICBSA_p', 'PCINECTA_p', 'MTFCC_p', 'FUNCSTAT_p', 'ALAND_p',
       'AWATER_p', 'INTPTLAT_p', 'INTPTLON_p', 'geometry_p'],
      dtype='object')

In [39]:
result_df_1.loc[result_df_1['GEOID_p'].isnull(),'GEOID_p'] = result_df_1['GEOID']
result_df_1

,CID,Community Name,County,STATEFP,COUNTYFP,GEOID,Counties,NAMELSAD,MATCH_SCORE,STATEFP_p,...,CLASSFP_p,PCICBSA_p,PCINECTA_p,MTFCC_p,FUNCSTAT_p,ALAND_p,AWATER_p,INTPTLAT_p,INTPTLON_p,geometry_p
0,10259.0,"ABBEVILLE, CITY OF",HENRY COUNTY,1,067,01067,Henry County,Abbeville city,88,1.0,...,C1,None,None,G4110,A,40255361.0,107642.0,+31.5647056,-085.2591213,"POLYGON ((-85.29580 31.55385, -85.29575 31.556..."
1,10097.0,"HEADLAND, CITY OF",HENRY COUNTY,1,067,01067,Henry County,Headland city,87,1.0,...,C1,None,None,G4110,A,78556180.0,25295.0,+31.3549740,-085.3570308,"POLYGON ((-85.41739 31.32953, -85.41734 31.333..."
2,10261.0,HENRY COUNTY*,HENRY COUNTY,1,067,01067,Henry County,<NA>,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,10260.0,"NEWVILLE, TOWN OF",HENRY COUNTY,1,067,01067,Henry County,Newville town,87,1.0,...,C1,None,None,G4110,A,10406795.0,0.0,+31.4206043,-085.3366900,"POLYGON ((-85.36864 31.42297, -85.36854 31.423..."
4,10413.0,"HALEBURG, TOWN OF",HENRY COUNTY,1,067,01067,Henry County,Haleburg town,87,1.0,...,C1,None,None,G4110,A,9940861.0,0.0,+31.4099758,-085.1380681,"POLYGON ((-85.15525 31.42360, -85.14784 31.423..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24858,480715.0,"QUITAQUE, CITY OF",BRISCOE COUNTY,48,045,48045,Briscoe County,Quitaque city,87,48.0,...,C1,None,None,G4110,A,1872755.0,0.0,+34.3674100,-101.0555975,"POLYGON ((-101.06479 34.36596, -101.06470 34.3..."
24859,510066.0,"GALAX, CITY OF",GALAX CITY,51,640,51640,Galax city,Galax city,83,51.0,...,C7,None,None,G4110,A,21332077.0,105290.0,+36.6656397,-080.9143083,"POLYGON ((-80.94982 36.65901, -80.94970 36.661..."
24860,510092.0,LOUISA COUNTY *,LOUISA COUNTY,51,109,51109,Louisa County,<NA>,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
24861,510378.0,"LOUISA, TOWN OF",LOUISA COUNTY,51,109,51109,Louisa County,Louisa town,85,51.0,...,C1,None,None,G4110,A,4706671.0,37057.0,+38.0166240,-077.9997371,"POLYGON ((-78.01780 38.02494, -78.01586 38.028..."


In [40]:
result_df_final = result_df_1[["CID", "Community Name", "County", "STATEFP", "COUNTYFP" , "GEOID", "Counties", "NAMELSAD", "PLACENS_p", "PLACEFP_p", "GEOID_p", "MATCH_SCORE"]].copy()

In [42]:
result_df_final

,CID,Community Name,County,STATEFP,COUNTYFP,GEOID,Counties,NAMELSAD,PLACENS_p,PLACEFP_p,GEOID_p,MATCH_SCORE
0,10259.0,"ABBEVILLE, CITY OF",HENRY COUNTY,1,067,01067,Henry County,Abbeville city,02403054,00124,0100124,88
1,10097.0,"HEADLAND, CITY OF",HENRY COUNTY,1,067,01067,Henry County,Headland city,02404672,33856,0133856,87
2,10261.0,HENRY COUNTY*,HENRY COUNTY,1,067,01067,Henry County,<NA>,NaN,NaN,01067,0
3,10260.0,"NEWVILLE, TOWN OF",HENRY COUNTY,1,067,01067,Henry County,Newville town,02406991,54600,0154600,87
4,10413.0,"HALEBURG, TOWN OF",HENRY COUNTY,1,067,01067,Henry County,Haleburg town,02406636,32656,0132656,87
...,...,...,...,...,...,...,...,...,...,...,...,...
24858,480715.0,"QUITAQUE, CITY OF",BRISCOE COUNTY,48,045,48045,Briscoe County,Quitaque city,02411509,60176,4860176,87
24859,510066.0,"GALAX, CITY OF",GALAX CITY,51,640,51640,Galax city,Galax city,01498426,30208,5130208,83
24860,510092.0,LOUISA COUNTY *,LOUISA COUNTY,51,109,51109,Louisa County,<NA>,NaN,NaN,51109,0
24861,510378.0,"LOUISA, TOWN OF",LOUISA COUNTY,51,109,51109,Louisa County,Louisa town,02391275,47144,5147144,85


Save files

In [44]:
result_df_final.to_csv(f"{data_dir}ReferenceDatasets/CountyCIDs.csv")

In [35]:
places_dfs.to_file(f"{data_dir}ReferenceDatasets/Places.shp")